# Filter and Merge New Rules

Takes 5m

This notebook:
1. Reads `bbo_bt_new_rules.parquet`
2. Filters criteria based on configurable thresholds (lift, pos count/pct, neg count/pct)
3. Creates `Accepted_Criteria` and `Rejected_Criteria` columns
4. Creates `Merged_Rules` by merging BT base criteria for the **correct seat** (`Agg_Expr_Seat_{seat}`) with accepted criteria
5. Ensures merged numeric inequalities (HCP / Total_Points / SL_*) do **not** contradict the BT base bounds
6. Create analytical reports and exports to Excel file.


In [2]:
# Configuration
import re
from pathlib import Path
import time
from typing import Dict, List, Tuple

import polars as pl

program_start_time = time.time()
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

# Paths
INPUT_FILE = Path("E:/bridge/data/bbo/bidding/bbo_bt_new_rules.parquet")
BT_SEAT1_FILE = Path("E:/bridge/data/bbo/bidding/bbo_bt_seat1.parquet")
OUTPUT_FILE = Path("E:/bridge/data/bbo/bidding/bbo_bt_merged_rules.parquet")

# =============================================================================
# THRESHOLD CONFIGURATION
# =============================================================================
# A criterion is ACCEPTED if ALL conditions are met:
#   - lift >= MIN_LIFT
#   - pos_rate >= MIN_POS_RATE (as decimal, e.g. 0.50 = 50%)
#   - pos_count >= MIN_POS_COUNT (absolute count based on pos_rate * group pos_count)
#   - neg_rate <= MAX_NEG_RATE (optional upper bound on negative rate)
#
# BUG FIX (2026-01-07): Changed MIN_POS_RATE from 0.10 to 0.50.
# The old 10% threshold accepted criteria like SL_S <= 1 for 1H openers even though
# only 21% of 1H hands have spade length <= 1. Since these criteria are enforced as
# HARD REQUIREMENTS in Bidding Arena, accepting criteria with low pos_rate causes
# 79%+ of valid hands to be incorrectly rejected. A 50% threshold ensures we only
# accept criteria that are true for the MAJORITY of positive cases.
#
# FIX #2 (2026-01-07): Added HIGH_CONFIDENCE_POS_RATE = 0.98 and logic to prefer
# the MOST RESTRICTIVE bound when multiple bounds pass the high-confidence threshold.
# Example: For 1H, both SL_H >= 4 (99.92%) and SL_H >= 5 (99.16%) pass 50%, but
# SL_H >= 5 is the correct requirement. Since SL_H >= 5 passes 98% and is more
# restrictive (higher value for >=), we should prefer it over SL_H >= 4.

MIN_LIFT = 1.5          # Minimum lift to accept criterion
MIN_POS_RATE = 0.50     # Minimum positive rate (50%) - must be true for majority of positives
MIN_POS_COUNT = 50      # Minimum absolute positive count for criterion
MAX_NEG_RATE = 0.80     # Maximum negative rate (reject if too common in negatives)
HIGH_CONFIDENCE_POS_RATE = 0.98  # Threshold for preferring most restrictive bound

# Sample auctions for inspection (reused across cells)
SAMPLE_AUCTIONS = ["1n", "1s", "1h", "1d", "1c"]

# Pre-compiled regex for parsing inequality criteria
INEQUALITY_RE = re.compile(r'(\w+)\s*(>=|<=|>|<|==|!=)\s*(\d+)')

print("Threshold Configuration:")
print(f"  MIN_LIFT: {MIN_LIFT}")
print(f"  MIN_POS_RATE: {MIN_POS_RATE:.0%}")
print(f"  MIN_POS_COUNT: {MIN_POS_COUNT}")
print(f"  MAX_NEG_RATE: {MAX_NEG_RATE:.0%}")
print(f"  HIGH_CONFIDENCE_POS_RATE: {HIGH_CONFIDENCE_POS_RATE:.0%}")

2026-01-07 11:04:39
Threshold Configuration:
  MIN_LIFT: 1.5
  MIN_POS_RATE: 50%
  MIN_POS_COUNT: 50
  MAX_NEG_RATE: 80%
  HIGH_CONFIDENCE_POS_RATE: 98%


In [3]:
# takes 3m45s
# Load data
print(f"Loading {INPUT_FILE}...")
if not INPUT_FILE.exists():
    raise FileNotFoundError(f"Input file not found: {INPUT_FILE}")

if not BT_SEAT1_FILE.exists():
    raise FileNotFoundError(f"BT seat-1 file not found: {BT_SEAT1_FILE}")

df = pl.read_parquet(INPUT_FILE)
print(f"  Loaded {df.height:,} rows")
print(f"  Columns: {df.columns}")

# Join in BT base criteria for the CORRECT SEAT.
# IMPORTANT: Some rows in bbo_bt_new_rules.parquet have base_rules=[], even for real auctions.
# We load ALL Agg_Expr_Seat_X columns and select the appropriate one based on each row's seat.
# 
# BUG FIX (2026-01-06): Previously we always used Agg_Expr_Seat_1, which caused opener's
# criteria (e.g., HCP >= 15) to be merged into other seats' Merged_Rules.
print(f"Loading BT base criteria from {BT_SEAT1_FILE}...")
bt = pl.read_parquet(BT_SEAT1_FILE, columns=[
    "bt_index", 
    "Agg_Expr_Seat_1", "Agg_Expr_Seat_2", "Agg_Expr_Seat_3", "Agg_Expr_Seat_4",
    "is_completed_auction", "Auction"
])
if "is_completed_auction" in bt.columns:
    bt = bt.filter(pl.col("is_completed_auction") == True)

bt = bt.select([
    pl.col("bt_index").cast(pl.Int64),
    pl.col("Agg_Expr_Seat_1").alias("bt_agg_seat_1"),
    pl.col("Agg_Expr_Seat_2").alias("bt_agg_seat_2"),
    pl.col("Agg_Expr_Seat_3").alias("bt_agg_seat_3"),
    pl.col("Agg_Expr_Seat_4").alias("bt_agg_seat_4"),
    pl.col("Auction").alias("bt_auction"),
])

# Normalize types and join
if "bt_index" in df.columns:
    df = df.with_columns(pl.col("bt_index").cast(pl.Int64))

if "base_rules" not in df.columns:
    df = df.with_columns(pl.lit([]).cast(pl.List(pl.Utf8)).alias("base_rules"))

# Join BT data
df = df.join(bt, on="bt_index", how="left")

# Select the correct Agg_Expr_Seat_X based on each row's seat column.
# This ensures we use seat 1's criteria for seat 1, seat 2's for seat 2, etc.
df = df.with_columns(
    pl.when(pl.col("seat") == 1)
    .then(pl.col("bt_agg_seat_1"))
    .when(pl.col("seat") == 2)
    .then(pl.col("bt_agg_seat_2"))
    .when(pl.col("seat") == 3)
    .then(pl.col("bt_agg_seat_3"))
    .when(pl.col("seat") == 4)
    .then(pl.col("bt_agg_seat_4"))
    .otherwise(pl.col("bt_agg_seat_1"))  # Fallback to seat 1 if seat is invalid
    .alias("bt_base_rules")
)

# Override base_rules with BT base when available.
df = df.with_columns(
    pl.when(
        pl.col("bt_base_rules").is_not_null() & (pl.col("bt_base_rules").list.len() > 0)
    )
    .then(pl.col("bt_base_rules"))
    .otherwise(pl.col("base_rules"))
    .alias("base_rules")
)

# Drop intermediate columns
df = df.drop(["bt_agg_seat_1", "bt_agg_seat_2", "bt_agg_seat_3", "bt_agg_seat_4", "bt_base_rules"])

print("Preview:")
print(df.select([c for c in ["step_auction", "seat", "bt_index", "bt_auction", "base_rules", "pos_count"] if c in df.columns]).head(5))



Loading E:\bridge\data\bbo\bidding\bbo_bt_new_rules.parquet...
  Loaded 66,814 rows
  Columns: ['step_auction', 'bt_index', 'seat', 'prefix', 'next_bid', 'pos_count', 'neg_count', 'base_rules', 'discovered_rules', 'criteria_with_metrics', 'New_Rules', 'top_lift', 'bt_row_found']
Loading BT base criteria from E:\bridge\data\bbo\bidding\bbo_bt_seat1.parquet...
Preview:
shape: (5, 6)
┌──────────────┬──────┬───────────┬────────────┬─────────────────────────────────┬───────────┐
│ step_auction ┆ seat ┆ bt_index  ┆ bt_auction ┆ base_rules                      ┆ pos_count │
│ ---          ┆ ---  ┆ ---       ┆ ---        ┆ ---                             ┆ ---       │
│ str          ┆ i64  ┆ i64       ┆ str        ┆ list[str]                       ┆ i64       │
╞══════════════╪══════╪═══════════╪════════════╪═════════════════════════════════╪═══════════╡
│ 1d           ┆ 1    ┆ 151867501 ┆ null       ┆ ["HCP <= 21", "HCP >= 11", … "… ┆ 3507593   │
│ 1c           ┆ 1    ┆ 0         ┆ null      

In [4]:
# Inspect criteria_with_metrics for sample auctions
print("Inspecting criteria_with_metrics for selected auctions:")
print("=" * 80)

for auc in SAMPLE_AUCTIONS:
    row = df.filter(pl.col("step_auction") == auc).to_dicts()
    if row:
        row = row[0]
        print(f"\n{auc} (seat {row['seat']}, {row['pos_count']:,} deals):")
        print(f"  Base rules: {row['base_rules']}")
        print(f"  Discovered criteria with metrics:")
        for m in row["criteria_with_metrics"]:
            lift_str = f"{m['lift']:.2f}" if m['lift'] is not None else "inf"
            print(f"    {m['criterion']:25s}  lift={lift_str:>8s}  pos={m['pos_rate']:>6.2%}  neg={m['neg_rate']:>6.2%}")


Inspecting criteria_with_metrics for selected auctions:

1n (seat 1, 2,518,162 deals):
  Base rules: ['HCP <= 17', 'HCP >= 15', 'SL_C <= 5', 'SL_C >= 2', 'SL_D <= 5', 'SL_D >= 2', 'SL_H <= 5', 'SL_H >= 2', 'SL_S <= 5', 'SL_S >= 2', 'Total_Points <= 18']
  Discovered criteria with metrics:
    HCP >= 15                  lift=    2.64  pos=87.78%  neg=33.23%
    HCP >= 16                  lift=    2.17  pos=54.79%  neg=25.29%
    HCP >= 14                  lift=    2.00  pos=96.92%  neg=48.43%
    Total_Points >= 16         lift=    1.85  pos=83.28%  neg=45.13%
    Total_Points >= 15         lift=    1.59  pos=96.38%  neg=60.44%
    Total_Points >= 17         lift=    1.54  pos=53.85%  neg=34.93%

1s (seat 1, 2,497,723 deals):
  Base rules: ['HCP <= 21', 'HCP >= 11', 'SL_S >= 5', 'Total_Points <= 22', 'Total_Points >= 12']
  Discovered criteria with metrics:
    SL_S >= 5                  lift=   22.26  pos=99.32%  neg= 4.46%
    SL_S >= 6                  lift=   13.55  pos=32.57%  neg=

## Threshold-Based Filtering

Filter criteria based on configured thresholds.


In [5]:
def filter_criteria(
    criteria_with_metrics: List[Dict],
    pos_count: int,
    min_lift: float = MIN_LIFT,
    min_pos_rate: float = MIN_POS_RATE,
    min_pos_count: int = MIN_POS_COUNT,
    max_neg_rate: float = MAX_NEG_RATE,
    high_confidence_pos_rate: float = HIGH_CONFIDENCE_POS_RATE,
) -> Tuple[List[str], List[str]]:
    """
    Filter criteria based on thresholds, preferring most restrictive high-confidence bounds.
    
    Args:
        criteria_with_metrics: List of dicts with 'criterion', 'lift', 'pos_rate', 'neg_rate'
        pos_count: Total positive count for the group (used to compute absolute criterion count)
        min_lift: Minimum lift threshold
        min_pos_rate: Minimum positive rate threshold
        min_pos_count: Minimum absolute positive count threshold
        max_neg_rate: Maximum negative rate threshold
        high_confidence_pos_rate: Threshold for preferring most restrictive bound (default 98%)
    
    Returns:
        (accepted_criteria, rejected_criteria)
        
    The function implements a two-phase acceptance strategy:
    1. First pass: accept all criteria that pass basic thresholds (lift, pos_rate >= 50%, etc.)
    2. Second pass: for each (variable, operator) group with multiple accepted criteria,
       select the MOST RESTRICTIVE one that still passes high_confidence_pos_rate (98%):
       - For >= (lower bounds): most restrictive = HIGHEST value
       - For <= (upper bounds): most restrictive = LOWEST value
       
    This prevents less restrictive but technically true criteria (like SL_H >= 4) from
    overshadowing the actual convention requirements (like SL_H >= 5).
    """
    if not criteria_with_metrics:
        return [], []
    
    # Phase 1: Determine which criteria pass basic thresholds
    # Keep track of both criterion string AND its pos_rate for phase 2
    passed_basic: List[Tuple[str, float]] = []  # (criterion, pos_rate)
    rejected = []
    
    for m in criteria_with_metrics:
        criterion = m["criterion"]
        lift = m.get("lift")  # None means infinity
        pos_rate = m.get("pos_rate", 0)
        neg_rate = m.get("neg_rate", 0)
        
        # Calculate absolute positive count for this criterion
        crit_pos_count = int(pos_rate * pos_count)
        
        # Check all conditions
        lift_ok = (lift is None) or (lift >= min_lift)
        pos_rate_ok = pos_rate >= min_pos_rate
        pos_count_ok = crit_pos_count >= min_pos_count
        neg_rate_ok = neg_rate <= max_neg_rate
        
        if lift_ok and pos_rate_ok and pos_count_ok and neg_rate_ok:
            passed_basic.append((criterion, pos_rate))
        else:
            rejected.append(criterion)
    
    if not passed_basic:
        return [], rejected
    
    # Phase 2: For each (variable, operator) group, select the most restrictive
    # bound that still passes the high-confidence threshold
    #
    # Logic:
    # - For >= (lower bounds): higher value is more restrictive
    #   E.g., SL_H >= 5 is more restrictive than SL_H >= 4
    # - For <= (upper bounds): lower value is more restrictive
    #   E.g., SL_S <= 3 is more restrictive than SL_S <= 5
    
    # Group numeric criteria by (variable, operator)
    # Key: (var_name, op) -> List of (value, pos_rate)
    numeric_groups: Dict[Tuple[str, str], List[Tuple[int, float]]] = {}
    non_numeric: List[str] = []
    
    for criterion, pos_rate in passed_basic:
        parsed = _parse_ineq_for_filter(criterion)
        if parsed:
            var_name, op, value = parsed
            key = (var_name, op)
            if key not in numeric_groups:
                numeric_groups[key] = []
            numeric_groups[key].append((value, pos_rate))
        else:
            # Non-numeric criterion - keep as-is
            non_numeric.append(criterion)
    
    # Select best bound for each group
    accepted = []
    
    for (var_name, op), candidates in numeric_groups.items():
        # Filter to high-confidence candidates
        high_conf = [(v, pr) for v, pr in candidates if pr >= high_confidence_pos_rate]
        
        if high_conf:
            # Select most restrictive among high-confidence candidates
            if op in ('>=', '>'):
                # Higher value is more restrictive for lower bounds
                best_value = max(v for v, _ in high_conf)
            else:  # <=, <
                # Lower value is more restrictive for upper bounds
                best_value = min(v for v, _ in high_conf)
            accepted.append(f"{var_name} {op} {best_value}")
        else:
            # No high-confidence candidate - fall back to least restrictive
            # (original behavior for low-confidence scenarios)
            if op in ('>=', '>'):
                best_value = min(v for v, _ in candidates)
            else:
                best_value = max(v for v, _ in candidates)
            accepted.append(f"{var_name} {op} {best_value}")
    
    # Add non-numeric criteria
    accepted.extend(non_numeric)
    
    return accepted, rejected


def _parse_ineq_for_filter(expr: str):
    """Parse inequality expression for filter_criteria. Returns (var_name, op, value) or None."""
    m = INEQUALITY_RE.match(expr.strip())
    if not m:
        return None
    name, op, value_str = m.groups()
    try:
        v = int(value_str)
    except ValueError:
        return None
    return name, op, v


## Merge / Deduplicate Numeric Criteria (Safe)

We observed that learned criteria can sometimes produce **contradictory** numeric bounds versus the BT base criteria
(e.g., `HCP <= 7` for `1S-p-p-p`).

In this notebook we treat `bt_seat1_df.Agg_Expr_Seat_1` as the **canonical base** constraints, then merge in
accepted learned criteria *additively*.

For numeric inequalities (e.g. `HCP`, `Total_Points`, `SL_S`):
- BT base numeric bounds are **canonical** (we do **not** tighten them with learned criteria)
- Accepted numeric bounds are only used when BT has **no numeric bound at all** for that metric
- When we do use accepted numeric bounds, we keep the **least restrictive** bound among accepted criteria
  (lower bounds: smallest value; upper bounds: largest value)

Relative comparisons (e.g. `SL_S >= SL_H`) are preserved as additional constraints.


In [6]:
# Merge / dedupe helpers
# 
# Key changes vs the previous implementation:
# - We treat BT seat-1 criteria as the canonical base (joined in Cell 2).
# - We merge ACCEPTED criteria *additively*.
# - For numeric inequalities (HCP / Total_Points / SL_*), we keep the tightest consistent bounds:
#     lower bound  -> max
#     upper bound  -> min
#   and we DROP any accepted bound that contradicts the base interval.
# 
# This prevents pathological learned constraints like HCP <= 7 for 1S openings.

def _parse_ineq(expr: str):
    m = INEQUALITY_RE.match(expr.strip())
    if not m:
        return None
    name, op, value = m.groups()
    try:
        v = int(value)
    except Exception:
        return None
    return name, op, v


def _bounds_from(criteria: List[str]) -> Tuple[Dict[str, int], Dict[str, int]]:
    lo: Dict[str, int] = {}
    hi: Dict[str, int] = {}
    for c in criteria or []:
        p = _parse_ineq(c)
        if not p:
            continue
        name, op, v = p
        if op in (">=", ">"):
            lo[name] = max(lo.get(name, v), v)
        elif op in ("<=", "<"):
            hi[name] = min(hi.get(name, v), v)
        elif op == "==":
            lo[name] = max(lo.get(name, v), v)
            hi[name] = min(hi.get(name, v), v)
    return lo, hi


def dedupe_criteria_least_restrictive(criteria: List[str]) -> List[str]:
    """Deduplicate criteria list, keeping least restrictive bounds for each variable.
    
    When multiple criteria refer to the same variable with the same operator:
    - For >= or >: keep the SMALLEST value (allows more hands through)
    - For <= or <: keep the LARGEST value (allows more hands through)
    
    Examples:
        ["HCP >= 3", "HCP >= 5", "HCP <= 10"]  -> ["HCP >= 3", "HCP <= 10"]
        ["SL_S >= 4", "SL_S >= 2", "Balanced"] -> ["SL_S >= 2", "Balanced"]
    """
    if not criteria:
        return []
    
    # Track inequalities: (var_name, operator) -> best_value
    inequalities: Dict[Tuple[str, str], int] = {}
    # Track non-numeric criteria (preserve order)
    other: List[str] = []
    
    for crit in criteria:
        crit_str = str(crit).strip()
        p = _parse_ineq(crit_str)
        if p:
            name, op, v = p
            key = (name, op)
            if key not in inequalities:
                inequalities[key] = v
            else:
                existing = inequalities[key]
                # For lower bounds (>=, >): keep smallest (least restrictive)
                if op in ('>=', '>'):
                    if v < existing:
                        inequalities[key] = v
                # For upper bounds (<=, <): keep largest (least restrictive)
                elif op in ('<=', '<'):
                    if v > existing:
                        inequalities[key] = v
                # For ==: first wins
        else:
            # Non-numeric criterion - add if not already present
            if crit_str not in other:
                other.append(crit_str)
    
    # Combine: inequalities first (sorted for consistency), then other
    result = [f"{name} {op} {v}" for (name, op), v in sorted(inequalities.items())]
    result.extend(other)
    return result


def merge_rules(base_rules: List[str], accepted_criteria: List[str]) -> List[str]:
    """Merge BT base_rules with accepted_criteria.

    UPDATED POLICY (for correctness + Arena matching):
    - BT numeric inequalities are canonical. If BT already constrains a numeric metric
      (HCP / Total_Points / SL_*), we do **not** tighten it with learned criteria.
      This prevents pathological tightening like adding `Total_Points <= 6` on top of
      a BT base `Total_Points <= 18`.
    - For numeric metrics that BT does *not* constrain at all, we may add learned bounds,
      but we keep the **least restrictive** bound among accepted criteria:
        - lower bounds (>=, >): keep the smallest value
        - upper bounds (<=, <): keep the largest value
    - Non-numeric accepted criteria (including relative SL comparisons) are appended.
    """
    base = [str(x).strip() for x in (base_rules or []) if str(x).strip()]
    accepted = [str(x).strip() for x in (accepted_criteria or []) if str(x).strip()]
    if not accepted:
        return base

    base_lo, base_hi = _bounds_from(base)
    base_constrained = set(base_lo.keys()) | set(base_hi.keys())

    kept_other: List[str] = []
    kept_eq_ne: List[str] = []
    add_lo: Dict[str, int] = {}
    add_hi: Dict[str, int] = {}

    for c in accepted:
        cs = str(c).strip()
        p = _parse_ineq(cs)
        if not p:
            kept_other.append(cs)
            continue

        name, op, v = p

        # Equality/inequality operators: keep verbatim
        if op in ("==", "!="):
            kept_eq_ne.append(cs)
            continue

        # Only treat these as numeric metrics; everything else is "other"
        if name not in ("HCP", "Total_Points") and not name.startswith("SL_"):
            kept_other.append(cs)
            continue

        # If BT already constrains this metric, do not add/tighten it
        if name in base_constrained:
            continue

        if op in (">=", ">"):
            add_lo[name] = min(add_lo.get(name, v), v)
        elif op in ("<=", "<"):
            add_hi[name] = max(add_hi.get(name, v), v)

    out: List[str] = []
    out.extend(base)

    for name in sorted(add_lo.keys()):
        out.append(f"{name} >= {add_lo[name]}")
    for name in sorted(add_hi.keys()):
        out.append(f"{name} <= {add_hi[name]}")

    # Add non-numeric criteria and eq/ne criteria (dedup preserving order)
    seen = set(out)
    for c in kept_other + kept_eq_ne:
        if c and c not in seen:
            out.append(c)
            seen.add(c)

    # Final deduplication: keep least restrictive bounds for each variable
    return dedupe_criteria_least_restrictive(out)


In [7]:
# Sanity tests (executable)
# Test filter_criteria
sample_row = df.filter(pl.col("step_auction") == "1n").to_dicts()[0]
acc, rej = filter_criteria(
    sample_row["criteria_with_metrics"],
    sample_row["pos_count"],
)
print("Test filter_criteria on '1n':")
print(f"  Accepted: {acc}")
print(f"  Rejected: {rej}")
print()

# Test merge_rules (do not tighten BT numeric bounds)
test_base = ["HCP >= 12", "HCP <= 21", "Total_Points >= 12", "Total_Points <= 22", "SL_S >= 5"]
# accepted contains a contradiction (HCP <= 7) and a tightening (HCP >= 13), but BT already constrains HCP,
# so numeric HCP bounds should NOT be tightened. Relative criteria should still be preserved.
test_accepted = ["HCP <= 7", "HCP >= 13", "SL_S >= SL_H"]
print("Test merge_rules (BT numeric bounds canonical):")
print(f"  Base rules: {test_base}")
print(f"  Accepted:   {test_accepted}")
print(f"  Merged:     {merge_rules(test_base, test_accepted)}")



Test filter_criteria on '1n':
  Accepted: ['HCP >= 14', 'Total_Points >= 15']
  Rejected: []

Test merge_rules (BT numeric bounds canonical):
  Base rules: ['HCP >= 12', 'HCP <= 21', 'Total_Points >= 12', 'Total_Points <= 22', 'SL_S >= 5']
  Accepted:   ['HCP <= 7', 'HCP >= 13', 'SL_S >= SL_H']
  Merged:     ['HCP <= 21', 'HCP >= 12', 'SL_S >= 5', 'Total_Points <= 22', 'Total_Points >= 12', 'SL_S >= SL_H']


## Apply to All Rows


In [8]:
# Process all rows
print("Processing all rows...")

# Convert to list of dicts for processing
rows = df.to_dicts()

accepted_list = []
rejected_list = []
merged_list = []

for row in rows:
    # Filter criteria based on thresholds
    accepted, rejected = filter_criteria(
        row.get("criteria_with_metrics", []),
        row.get("pos_count", 0),
    )
    
    # Merge base_rules with accepted criteria (with deduplication)
    merged = merge_rules(
        row.get("base_rules", []),
        accepted
    )
    
    accepted_list.append(accepted)
    rejected_list.append(rejected)
    merged_list.append(merged)

# Add new columns
result_df = df.with_columns([
    pl.Series("Accepted_Criteria", accepted_list),
    pl.Series("Rejected_Criteria", rejected_list),
    pl.Series("Merged_Rules", merged_list),
])

print(f"  Processed {result_df.height:,} rows")
print(f"  New columns: Accepted_Criteria, Rejected_Criteria, Merged_Rules")


Processing all rows...
  Processed 66,814 rows
  New columns: Accepted_Criteria, Rejected_Criteria, Merged_Rules


In [9]:
# Preview results
print("Preview of results:")
display_cols = ["step_auction", "pos_count", "base_rules", "Accepted_Criteria", "Rejected_Criteria", "Merged_Rules"]
print(result_df.select([c for c in display_cols if c in result_df.columns]).head(20))


Preview of results:
shape: (20, 6)
┌──────────────┬───────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ step_auction ┆ pos_count ┆ base_rules      ┆ Accepted_Criter ┆ Rejected_Criter ┆ Merged_Rules    │
│ ---          ┆ ---       ┆ ---             ┆ ia              ┆ ia              ┆ ---             │
│ str          ┆ i64       ┆ list[str]       ┆ ---             ┆ ---             ┆ list[str]       │
│              ┆           ┆                 ┆ list[str]       ┆ list[str]       ┆                 │
╞══════════════╪═══════════╪═════════════════╪═════════════════╪═════════════════╪═════════════════╡
│ 1d           ┆ 3507593   ┆ ["HCP <= 21",   ┆ ["SL_D >= 3"]   ┆ ["SL_D >= 6",   ┆ ["HCP <= 21",   │
│              ┆           ┆ "HCP >= 11", …  ┆                 ┆ "SL_S <= 1",    ┆ "HCP >= 11", …  │
│              ┆           ┆ "…              ┆                 ┆ "SL…            ┆ "…              │
│ 1c           ┆ 3446023   ┆ ["HCP <= 21",   ┆ ["SL_C >=

In [10]:
# Detailed inspection of sample auctions
print("Detailed inspection of sample auctions:")
print("=" * 100)

for auc in SAMPLE_AUCTIONS:
    rows = result_df.filter(pl.col("step_auction") == auc).to_dicts()
    if rows:
        row = rows[0]
        print(f"\n{auc} (seat {row['seat']}, {row['pos_count']:,} pos, {row['neg_count']:,} neg):")
        print(f"  Base rules:        {row['base_rules']}")
        print(f"  Accepted criteria: {row['Accepted_Criteria']}")
        print(f"  Rejected criteria: {row['Rejected_Criteria']}")
        print(f"  Merged rules:      {row['Merged_Rules']}")
        
        # Show criteria with metrics
        print(f"  Criteria details:")
        for m in row.get("criteria_with_metrics", []):
            lift_str = f"{m['lift']:.2f}" if m['lift'] is not None else "inf"
            status = "✓" if m['criterion'] in row['Accepted_Criteria'] else "✗"
            print(f"    {status} {m['criterion']:25s}  lift={lift_str:>8s}  pos={m['pos_rate']:>6.2%}  neg={m['neg_rate']:>6.2%}")


Detailed inspection of sample auctions:

1n (seat 1, 2,518,162 pos, 13,476,665 neg):
  Base rules:        ['HCP <= 17', 'HCP >= 15', 'SL_C <= 5', 'SL_C >= 2', 'SL_D <= 5', 'SL_D >= 2', 'SL_H <= 5', 'SL_H >= 2', 'SL_S <= 5', 'SL_S >= 2', 'Total_Points <= 18']
  Accepted criteria: ['HCP >= 14', 'Total_Points >= 15']
  Rejected criteria: []
  Merged rules:      ['HCP <= 17', 'HCP >= 15', 'SL_C <= 5', 'SL_C >= 2', 'SL_D <= 5', 'SL_D >= 2', 'SL_H <= 5', 'SL_H >= 2', 'SL_S <= 5', 'SL_S >= 2', 'Total_Points <= 18']
  Criteria details:
    ✗ HCP >= 15                  lift=    2.64  pos=87.78%  neg=33.23%
    ✗ HCP >= 16                  lift=    2.17  pos=54.79%  neg=25.29%
    ✓ HCP >= 14                  lift=    2.00  pos=96.92%  neg=48.43%
    ✗ Total_Points >= 16         lift=    1.85  pos=83.28%  neg=45.13%
    ✓ Total_Points >= 15         lift=    1.59  pos=96.38%  neg=60.44%
    ✗ Total_Points >= 17         lift=    1.54  pos=53.85%  neg=34.93%

1s (seat 1, 2,497,723 pos, 13,497,104 n

## Statistics


In [11]:
# Acceptance statistics
print("Acceptance Statistics:")
print("=" * 60)

# Count accepted/rejected per row
stats_df = result_df.with_columns([
    pl.col("Accepted_Criteria").list.len().alias("n_accepted"),
    pl.col("Rejected_Criteria").list.len().alias("n_rejected"),
    pl.col("Merged_Rules").list.len().alias("n_merged"),
    pl.col("base_rules").list.len().alias("n_base"),
    pl.col("discovered_rules").list.len().alias("n_discovered"),
])

print(f"\nRows with at least 1 accepted criterion: {stats_df.filter(pl.col('n_accepted') > 0).height:,}")
print(f"Rows with no accepted criteria: {stats_df.filter(pl.col('n_accepted') == 0).height:,}")

print("\nDistribution of accepted criteria count:")
print(stats_df.group_by("n_accepted").agg(pl.len().alias("count")).sort("n_accepted"))

print("\nDistribution of rejected criteria count:")
print(stats_df.group_by("n_rejected").agg(pl.len().alias("count")).sort("n_rejected"))

Acceptance Statistics:

Rows with at least 1 accepted criterion: 60,351
Rows with no accepted criteria: 6,463

Distribution of accepted criteria count:
shape: (7, 2)
┌────────────┬───────┐
│ n_accepted ┆ count │
│ ---        ┆ ---   │
│ u64        ┆ u64   │
╞════════════╪═══════╡
│ 0          ┆ 6463  │
│ 1          ┆ 15543 │
│ 2          ┆ 30103 │
│ 3          ┆ 10710 │
│ 4          ┆ 3437  │
│ 5          ┆ 522   │
│ 6          ┆ 36    │
└────────────┴───────┘

Distribution of rejected criteria count:
shape: (7, 2)
┌────────────┬───────┐
│ n_rejected ┆ count │
│ ---        ┆ ---   │
│ u64        ┆ u64   │
╞════════════╪═══════╡
│ 0          ┆ 8581  │
│ 1          ┆ 4083  │
│ 2          ┆ 9347  │
│ 3          ┆ 13718 │
│ 4          ┆ 15072 │
│ 5          ┆ 10976 │
│ 6          ┆ 5037  │
└────────────┴───────┘


In [12]:
# Most commonly accepted criteria
print("\nTop 20 most commonly accepted criteria:")
all_accepted = result_df.select(pl.col("Accepted_Criteria").explode().alias("criterion")).drop_nulls()
accepted_counts = all_accepted.group_by("criterion").agg(pl.len().alias("count")).sort("count", descending=True)
print(accepted_counts.head(20))



Top 20 most commonly accepted criteria:
shape: (20, 2)
┌────────────────────┬───────┐
│ criterion          ┆ count │
│ ---                ┆ ---   │
│ str                ┆ u64   │
╞════════════════════╪═══════╡
│ Total_Points <= 14 ┆ 4030  │
│ Total_Points <= 15 ┆ 3837  │
│ HCP <= 12          ┆ 3509  │
│ HCP <= 13          ┆ 3195  │
│ Total_Points <= 13 ┆ 3164  │
│ …                  ┆ …     │
│ SL_H >= 4          ┆ 1977  │
│ SL_C >= 3          ┆ 1951  │
│ SL_D >= 4          ┆ 1945  │
│ Total_Points <= 9  ┆ 1942  │
│ SL_S <= 2          ┆ 1941  │
└────────────────────┴───────┘


In [13]:
# Most commonly rejected criteria
print("\nTop 20 most commonly rejected criteria:")
all_rejected = result_df.select(pl.col("Rejected_Criteria").explode().alias("criterion")).drop_nulls()
rejected_counts = all_rejected.group_by("criterion").agg(pl.len().alias("count")).sort("count", descending=True)
print(rejected_counts.head(20))



Top 20 most commonly rejected criteria:
shape: (20, 2)
┌────────────────────┬───────┐
│ criterion          ┆ count │
│ ---                ┆ ---   │
│ str                ┆ u64   │
╞════════════════════╪═══════╡
│ Total_Points <= 13 ┆ 6939  │
│ HCP <= 11          ┆ 5248  │
│ SL_C <= 1          ┆ 4534  │
│ SL_D <= 1          ┆ 4391  │
│ HCP <= 12          ┆ 4205  │
│ …                  ┆ …     │
│ Total_Points >= 18 ┆ 3070  │
│ HCP <= 5           ┆ 3053  │
│ Total_Points <= 10 ┆ 3045  │
│ SL_C >= 4          ┆ 3037  │
│ Total_Points >= 19 ┆ 3029  │
└────────────────────┴───────┘


In [14]:
# Deduplication impact
print("\nDeduplication Impact:")
print("=" * 60)

# Compare: base_rules + accepted vs merged (after dedup)
dedup_stats = stats_df.with_columns([
    (pl.col("n_base") + pl.col("n_accepted")).alias("n_before_dedup"),
]).with_columns([
    (pl.col("n_before_dedup") - pl.col("n_merged")).alias("n_removed_by_dedup"),
])

total_before = dedup_stats.select(pl.col("n_before_dedup").sum()).item()
total_after = dedup_stats.select(pl.col("n_merged").sum()).item()
total_removed = dedup_stats.select(pl.col("n_removed_by_dedup").sum()).item()

print(f"Total criteria before dedup: {total_before:,}")
print(f"Total criteria after dedup:  {total_after:,}")
if total_before > 0:
    print(f"Criteria removed by dedup:   {total_removed:,} ({100*total_removed/total_before:.1f}%)")

# Rows where dedup had an effect
affected_rows = dedup_stats.filter(pl.col("n_removed_by_dedup") > 0).height
print(f"\nRows affected by deduplication: {affected_rows:,}")



Deduplication Impact:
Total criteria before dedup: 447,004
Total criteria after dedup:  363,818
Criteria removed by dedup:   83,186 (18.6%)

Rows affected by deduplication: 54,478


## Save Output


In [15]:
# Select final columns to save
final_cols = [
    "step_auction",
    "bt_index",
    "seat",
    "prefix",
    "next_bid",
    "pos_count",
    "neg_count",
    "base_rules",
    "discovered_rules",
    "criteria_with_metrics",
    "Accepted_Criteria",
    "Rejected_Criteria",
    "Merged_Rules",
    "top_lift",
    "bt_row_found",
]

# Filter to existing columns
save_cols = [c for c in final_cols if c in result_df.columns]
output_df = result_df.select(save_cols)

# Ensure output directory exists
OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)

print(f"Saving to {OUTPUT_FILE}...")
output_df.write_parquet(OUTPUT_FILE)

file_size_mb = OUTPUT_FILE.stat().st_size / (1024 * 1024)
print(f"  Saved {output_df.height:,} rows")
print(f"  File size: {file_size_mb:.2f} MB")


Saving to E:\bridge\data\bbo\bidding\bbo_bt_merged_rules.parquet...
  Saved 66,814 rows
  File size: 8.83 MB


In [16]:
print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"\nThresholds used:")
print(f"  MIN_LIFT: {MIN_LIFT}")
print(f"  MIN_POS_RATE: {MIN_POS_RATE:.0%}")
print(f"  MIN_POS_COUNT: {MIN_POS_COUNT}")
print(f"  MAX_NEG_RATE: {MAX_NEG_RATE:.0%}")
print(f"  HIGH_CONFIDENCE_POS_RATE: {HIGH_CONFIDENCE_POS_RATE:.0%}")
print(f"\nResults:")
print(f"  Total rows: {result_df.height:,}")
print(f"  Rows with accepted criteria: {stats_df.filter(pl.col('n_accepted') > 0).height:,}")
print(f"  Total accepted criteria: {all_accepted.height:,}")
print(f"  Total rejected criteria: {all_rejected.height:,}")
print(f"  Criteria removed by dedup: {total_removed:,}")
print(f"\nOutput: {OUTPUT_FILE}")



SUMMARY

Thresholds used:
  MIN_LIFT: 1.5
  MIN_POS_RATE: 50%
  MIN_POS_COUNT: 50
  MAX_NEG_RATE: 80%
  HIGH_CONFIDENCE_POS_RATE: 98%

Results:
  Total rows: 66,814
  Rows with accepted criteria: 60,351
  Total accepted criteria: 124,453
  Total rejected criteria: 209,321
  Criteria removed by dedup: 83,186

Output: E:\bridge\data\bbo\bidding\bbo_bt_merged_rules.parquet


In [20]:
# =============================================================================
# EXPORT ANALYSIS REPORT
# =============================================================================
from datetime import datetime
import pandas as pd

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
REPORT_DIR = Path(".")  # Current directory
REPORT_XLSX = REPORT_DIR / f"bbo_bt_filter_analysis_{timestamp}.xlsx"

print("=" * 80)
print("EXPORT ANALYSIS REPORT")
print("=" * 80)

# Prepare summary DataFrame
summary_data = {
    "Metric": [
        "MIN_LIFT", "MIN_POS_RATE", "MIN_POS_COUNT", "MAX_NEG_RATE", "HIGH_CONFIDENCE_POS_RATE",
        "Total Rows", "Rows with Accepted Criteria", "Total Accepted", "Total Rejected", "Removed by Dedup"
    ],
    "Value": [
        str(MIN_LIFT), f"{MIN_POS_RATE:.0%}", str(MIN_POS_COUNT), f"{MAX_NEG_RATE:.0%}", f"{HIGH_CONFIDENCE_POS_RATE:.0%}",
        str(result_df.height), str(stats_df.filter(pl.col('n_accepted') > 0).height),
        str(all_accepted.height), str(all_rejected.height), str(total_removed)
    ]
}
summary_df = pl.DataFrame(summary_data)

# Accepted criteria with details - sorted by auction
accepted_details_df = result_df.select([
    "step_auction", "seat", "prefix", "next_bid",
    pl.col("Accepted_Criteria").alias("criteria")
]).explode("criteria").drop_nulls().sort("step_auction")

if "criteria_with_metrics" in result_df.columns:
    metrics_df = result_df.select([
        "step_auction",
        pl.col("criteria_with_metrics").alias("metrics")
    ]).explode("metrics").drop_nulls()
    
    if metrics_df.height > 0:
        metrics_parsed = metrics_df.with_columns([
            pl.col("metrics").struct.field("criterion").alias("criteria"),
            pl.col("metrics").struct.field("pos_rate").alias("pos_rate"),
            pl.col("metrics").struct.field("neg_rate").alias("neg_rate"),
            pl.col("metrics").struct.field("lift").alias("lift"),
        ]).drop("metrics")
        
        accepted_details_df = accepted_details_df.join(
            metrics_parsed, on=["step_auction", "criteria"], how="left"
        ).sort("step_auction")

# Rejected criteria with details - sorted by auction
rejected_details_df = result_df.select([
    "step_auction", "seat", "prefix", "next_bid",
    pl.col("Rejected_Criteria").alias("criteria")
]).explode("criteria").drop_nulls().sort("step_auction")

# New rules (criteria in Merged but not in base) - sorted by auction
new_rules_df_export = result_df.select([
    "step_auction", "seat", "prefix", "next_bid",
    pl.col("Merged_Rules").alias("merged"),
    pl.col("base_rules").alias("base"),
]).with_columns([
    pl.col("merged").list.set_difference(pl.col("base")).alias("new_rules")
]).explode("new_rules").drop_nulls().select([
    "step_auction", "seat", "prefix", "next_bid", "new_rules"
]).sort("step_auction")

# Per-auction summary - sorted by auction
auction_summary = stats_df.select([
    "step_auction", "n_base", "n_discovered", "n_accepted", "n_rejected", "n_merged"
]).sort("step_auction")

# Helper function to autosize columns
def autosize_columns(writer, sheet_name, df_pandas):
    """Set column widths to max of header and longest value."""
    worksheet = writer.sheets[sheet_name]
    for idx, col in enumerate(df_pandas.columns):
        # Get max length of column name and values
        max_len = len(str(col))
        if len(df_pandas) > 0:
            # Sample up to 10000 rows for performance
            sample = df_pandas[col].head(10000).astype(str)
            max_val_len = sample.str.len().max()
            max_len = max(max_len, max_val_len)
        # Add a little padding and cap at reasonable width
        worksheet.set_column(idx, idx, min(max_len + 2, 50))

# Write to Excel with multiple sheets using pandas
with pd.ExcelWriter(REPORT_XLSX, engine="xlsxwriter") as writer:
    # Summary sheet
    summary_pd = summary_df.to_pandas()
    summary_pd.to_excel(writer, sheet_name="Summary", index=False)
    autosize_columns(writer, "Summary", summary_pd)
    
    # Accepted criteria sheet
    if accepted_details_df.height > 0:
        accepted_pd = accepted_details_df.to_pandas()
        accepted_pd.to_excel(writer, sheet_name="Accepted_Criteria", index=False)
        autosize_columns(writer, "Accepted_Criteria", accepted_pd)
    
    # Rejected criteria sheet
    if rejected_details_df.height > 0:
        rejected_pd = rejected_details_df.to_pandas()
        rejected_pd.to_excel(writer, sheet_name="Rejected_Criteria", index=False)
        autosize_columns(writer, "Rejected_Criteria", rejected_pd)
    
    # New rules sheet
    if new_rules_df_export.height > 0:
        new_rules_pd = new_rules_df_export.to_pandas()
        new_rules_pd.to_excel(writer, sheet_name="New_Rules", index=False)
        autosize_columns(writer, "New_Rules", new_rules_pd)
    
    # Auction summary sheet
    auction_pd = auction_summary.to_pandas()
    auction_pd.to_excel(writer, sheet_name="Auction_Summary", index=False)
    autosize_columns(writer, "Auction_Summary", auction_pd)

print(f"\n✓ Excel report saved: {REPORT_XLSX.absolute()}")
print(f"\n  Sheets created (all sorted by auction):")
print(f"    1. Summary - {summary_df.height} metrics")
print(f"    2. Accepted_Criteria - {accepted_details_df.height:,} criteria")
print(f"    3. Rejected_Criteria - {rejected_details_df.height:,} criteria")
print(f"    4. New_Rules - {new_rules_df_export.height:,} new rules")
print(f"    5. Auction_Summary - {auction_summary.height:,} auctions")

print("\n" + "=" * 80)

EXPORT ANALYSIS REPORT

✓ Excel report saved: c:\sw\bridge\ML-Contract-Bridge\src\BBO_GIB_Bidding_Playground\bbo_bt_filter_analysis_20260107_112217.xlsx

  Sheets created (all sorted by auction):
    1. Summary - 10 metrics
    2. Accepted_Criteria - 124,453 criteria
    3. Rejected_Criteria - 209,321 criteria
    4. New_Rules - 41,267 new rules
    5. Auction_Summary - 66,814 auctions



In [21]:
print('Program elapsed time in seconds:', time.time()-program_start_time)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

Program elapsed time in seconds: 1074.4085602760315
2026-01-07 11:22:33
